<a href="https://colab.research.google.com/github/szymanski52/multi-label-classification/blob/main/multi_label_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
!pip3 install kaggle==1.5.4
from google.colab import files
 
uploaded = files.upload()
 
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d shivanandmn/multilabel-classification-dataset
!unzip multilabel-classification-dataset.zip
!pip3 install pytorch-nlp
!pip3 install transformers

     |████████████████████████████████| 61kB 5.4MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.4-cp37-none-any.whl size=65139 sha256=d3b857f37e2adac6b248868515bc29b524a46ae523054288c3e04fed23d203b4
  Stored in directory: /root/.cache/pip/wheels/87/ea/09/173986e395d051411b9d547a69fe96cdc26208cb1bcc3e5567
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 68 bytes
 61% 7.00M/11.4M [00:00<00:00, 67.2MB/s]
100% 11.4M/11.4M [00:00<00:00, 64.9MB/s]
Archive:  multilabel-classification-dataset.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
     |████████████████████████████████| 92kB 7.6MB/s 
     |████████████████████████████████| 2.2MB 17.0MB/s 
     |████████████████████████████████| 3.3MB 41.1MB/s 
     |████████████████████████████████| 870kB 55.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=c312b885d5693a03885c402ea2026f492311a249067ae2ad50c1e44e22de78e3
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from imblearn.over_sampling import RandomOverSampler
from transformers import BertTokenizer
from collections import Counter
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('train.csv')

In [ ]:
data

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
20967,20968,Contemporary machine learning: a guide for pra...,Machine learning is finding increasingly bro...,1,1,0,0,0,0
20968,20969,Uniform diamond coatings on WC-Co hard alloy c...,Polycrystalline diamond coatings have been g...,0,1,0,0,0,0
20969,20970,Analysing Soccer Games with Clustering and Con...,We present a new approach for identifying si...,1,0,0,0,0,0
20970,20971,On the Efficient Simulation of the Left-Tail o...,The sum of Log-normal variates is encountere...,0,0,1,1,0,0


In [ ]:
print(Counter(data['Computer Science']))
print(Counter(data['Physics']))
print(Counter(data['Mathematics']))
print(Counter(data['Statistics']))
print(Counter(data['Quantitative Biology']))
print(Counter(data['Quantitative Finance']))

Counter({0: 12378, 1: 8594})
Counter({0: 14959, 1: 6013})
Counter({0: 15354, 1: 5618})
Counter({0: 15766, 1: 5206})
Counter({0: 20385, 1: 587})
Counter({0: 20723, 1: 249})


In [ ]:
def tokenize(tokenizer, data):
    title_ids = [tokenizer.encode(i, add_special_tokens = True, padding='max_length', truncation=True, max_length=20) for i in data['TITLE'].to_list()]
    abstract_ids = [tokenizer.encode(i, add_special_tokens = True, padding='max_length', truncation=True, max_length=200) for i in data['ABSTRACT'].to_list()]
    ids = []
    for i, j in zip(title_ids, abstract_ids):
        ids.append(i + j)
 
    mask = []
    title_mask = ((np.array(title_ids)>0)*1).tolist()
    abstract_mask = ((np.array(abstract_ids)>0)*1).tolist()
    for i, j in zip(title_mask, abstract_mask):
        mask.append(i + j)
 
    data['text_vector'] = ids
    data['mask'] = mask

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
 
tokenize(data)

In [ ]:
ros = RandomOverSampler()
 
columns = ['ID', 'TITLE', 'ABSTRACT', 'Computer Science',
           'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology',
           'Quantitative Finance', 'text_vector', 'mask']
 
#oversampling columns 'Quantitative Biology' and 'Quantitative Finance'
for i in columns[7:9]:
    data_y = data[i]
    data_X = data.drop(i, axis=1)
    X_ros, y_ros = ros.fit_sample(data_X, data_y)
    columns.remove(i)
    data = pd.DataFrame(X_ros, columns=columns)
    data[i] = y_ros
    columns.append(i)

In [ ]:
print(Counter(data['Computer Science']))
print(Counter(data['Physics']))
print(Counter(data['Mathematics']))
print(Counter(data['Statistics']))
print(Counter(data['Quantitative Biology']))
print(Counter(data['Quantitative Finance']))

Counter({0: 69812, 1: 10992})
Counter({0: 74791, 1: 6013})
Counter({0: 75081, 1: 5723})
Counter({0: 68934, 1: 11870})
Counter({0: 47235, 1: 33569})
Counter({0: 40402, 1: 40402})


In [ ]:
labels = data[['Computer Science', 'Physics',
               'Mathematics', 'Statistics',
               'Quantitative Biology', 'Quantitative Finance']].values.tolist()

data['labels'] = labels

In [ ]:
class DS(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
 
    def __len__(self):
        return len(self.dataframe)
 
    def __getitem__(self, index):
        return self.dataframe.iloc[index]

In [ ]:
df = data[['ID', 'text_vector', 'mask', 'labels']]

In [ ]:
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(df, test_size=0.1)
train_ds = DS(train_data)
val_ds = DS(val_data)

In [ ]:
def gen_batch(batch):
    text_vectors = torch.Tensor([i[1] for i in batch]).long()
    labels = torch.Tensor([i[3] for i in batch]).long()
    att_mask = torch.Tensor([i[2] for i in batch]).long()
    return text_vectors, labels, att_mask

In [ ]:
train_dl = DataLoader(train_ds, batch_size=64, collate_fn=gen_batch, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=64, collate_fn=gen_batch, shuffle=True)

In [ ]:
from transformers import AutoModel, AdamW
 
bert = AutoModel.from_pretrained('bert-base-uncased')
for param in bert.parameters():
    param.requires_grad = False

In [ ]:
class BertClassifier(nn.Module):
    def __init__(self, bert):
        super(BertClassifier, self).__init__()
        self.bert = bert 
        self.dropout = nn.Dropout(0.1)
        self.relu =  nn.ReLU()
        self.fc1 = nn.Linear(768,256)
        self.fc2 = nn.Linear(256,6)
        self.sigmoid = nn.Sigmoid()

    def forward(self, sent_id, mask):
        bert_output = self.bert(sent_id, attention_mask=mask)
        pooler_out = bert_output.pooler_output
        x = self.fc1(pooler_out)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [ ]:
model = BertClassifier(bert)
model.cuda()

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [ ]:
from sklearn.metrics import f1_score
def train(data):
    train_loss = 0
    train_acc = 0
    for i, (vector, label, att_mask) in enumerate(data):
        optimizer.zero_grad()
        vector, label, att_mask = vector.to(device), label.to(device), att_mask.to(device)
        output = model(vector, att_mask)
        loss = criterion(output, label.float())
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.round() == label).sum().item()
    scheduler.step()
    return train_loss / len(data), train_acc / len(data)

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    with torch.no_grad():
        for idx, (vector, label, att_mask) in enumerate(dataloader):
            vector, label, att_mask = vector.to(device), label.to(device), att_mask.to(device)
            predited_label = model(vector, att_mask)
            loss = criterion(predited_label, label.float())
            total_acc += (predited_label.round() == label).sum().item()
            total_count += label.size(0)
    return (total_acc/total_count/6, f1_score(label.cpu(), (predited_label >= 0.5).cpu(), average='micro'))

In [ ]:
import time
from torch.utils.data.dataset import random_split
# Hyperparameters
EPOCHS = 10
LR = 1e-3  # learning rate
  
criterion = torch.nn.BCELoss()
optimizer = AdamW(model.parameters(), lr = LR, eps = 1e-8)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1e-2, gamma=1e-6)
total_accuracy = None
device = torch.device('cuda:0')
 
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dl)
    val_scores = evaluate(val_dl)
    accu_val = val_scores[0]
    f1_val = val_scores[1]
    if total_accuracy is not None and total_accuracy > accu_val:
        scheduler.step()
    else:
        total_accuracy = accu_val
    print('-' * 90)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} |  valid micro-f1 {:8.3f} |'.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val, f1_val))
    print('-' * 90)

------------------------------------------------------------------------------------------
| end of epoch   1 | time: 1153.29s | valid accuracy    0.924 |  valid micro-f1    0.821 |
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
| end of epoch   2 | time: 1159.00s | valid accuracy    0.939 |  valid micro-f1    0.791 |
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
| end of epoch   3 | time: 1158.12s | valid accuracy    0.944 |  valid micro-f1    0.900 |
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
| end of epoch   4 | time: 1157.73s | valid accuracy    0.946 |  valid micro-f1    0.872 |

In [ ]:
test_data = pd.read_csv('test.csv')
tokenize(tokenizer, test_data)

In [ ]:
def gen_test_batch(batch):
    ids = torch.Tensor([i[0] for i in batch]).long()
    text_vectors = torch.Tensor([i[3] for i in batch]).long()
    att_mask = torch.Tensor([i[4] for i in batch]).long()
    return ids, text_vectors, att_mask

In [ ]:
test_ds = DS(test_data)
test_dl = DataLoader(test_ds, batch_size=16, collate_fn=gen_test_batch, shuffle=False)
pred = []
for i, (ids, vector, att_mask) in enumerate(test_dl):
    ids, vector, att_mask = ids.to(device), vector.to(device), att_mask.to(device)
    output = model(vector, att_mask)
    out_df = torch.cat((ids.unsqueeze(1), (output >= 0.5) * 1), 1)
    pred += out_df

In [ ]:
import csv
with open('submission.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['ID','Computer Science', 'Physics',
                     'Mathematics', 'Statistics',
                     'Quantitative Biology', 'Quantitative Finance'])
    for i in pred:
        writer.writerow(i.tolist())

In [ ]:
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>